In [48]:
import dateutil.parser as parser
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse
import re
import datetime

In [49]:
def scoring (news):
    article = news.Article
    createdDate = news.CreatedDate
    title = news.Title
    author = news.Author
    originalContent = news.OriginalContent
    lastUsed = news.LastUsed
    href = news.ArticleUrl
    fetchedDate = news.FetchedDate
    
    print (href)
    
    score = 0
    
    if (title):
        score += 1
        
    if (article):
        list_number = []
        score_para = 0
        for paragraphs in article:
            length = len(re.findall(r'\w+', paragraphs))
            list_number.append(length)
        max_number = max(list_number)
#         print (max_number)
        if (max_number <= 100):
            score += 1
    
    if (createdDate):
        createdDate_str = datetime.datetime.strptime(createdDate, "%Y-%m-%dT%H:%M:%S")
        fetchedDate_str = datetime.datetime.strptime(fetchedDate, "%Y-%m-%dT%H:%M:%S")    
        duration = fetchedDate_str - createdDate_str
         if (0 < duration.days <= 90):
                score += 5
            elif (91 <= duration.days <= 182):
                score += 4
            elif (183 <= duration.days <= 273):
                score += 3
            elif (274 <= duration.days <= 365):
                score += 2
            else:
                score += 1
        
    if (author):
        score += 1
    
    if (lastUsed):
        score += 1
    
    if (fetchedDate):
        score += 1
        
    print ('********total:**********', score)

In [ ]:

class News:
    Article = []
    Title = ''
    Author = ''
    OriginalContent = ''
    CreatedDate = ''
    FetchedDate = ''
    ArticleUrl = ''
    LastUsed = ''
    
    def __init__(self, article , title, author, originalcontent, createddate, fetcheddate, articleurl, lastused):
        self.Article = article
        self.Title = title
        self.OriginalContent = originalcontent
        self.CreatedDate = createddate
        self.FetchedDate = fetcheddate
        self.ArticleUrl = articleurl
        self.LastUsed = lastused
        self.Author = author


def get_news_firstpage(link):
    for i in range(len(link)):
        link[i] = 'http://' + link[i]
        headers = {'user-agent' : 'Mozilla/5.0'}            
        try:
            source = requests.get(link[i], headers = headers)
            soup = BeautifulSoup(source.content, "lxml")
        except requests.exceptions.ConnectionError:
            print ('Connection refused!!!')
        
        
        if (soup.findAll('article')):
            for article in soup.findAll('article'):
                if (article.find('a')):
                    href = article.find('a').get('href')
                    if href.startswith('http'):
                        href = href
                    else:
                        href = link[i] + href
                    get_news_content(href)
                else:
                    print ('not able to get <a>!!')
        else: 
            print ('not able to get <article>!!')
        
    
 
def get_news_content(href):
    headers = {'user-agent' : 'Mozilla/5.0'}
    try:
        source = requests.get(href, headers = headers)
    except requests.exceptions.ConnectionError:
        source.status_code = "Connection refused"
    if (BeautifulSoup(source.content, "lxml").body):
        soup = BeautifulSoup(source.content, "lxml").body
        if (soup.findAll('p')):            
            article = []
            for paragraph in soup.findAll('p'):
                article.append(paragraph.text)
        else:
            article = 'Unknown'
        date = time.strftime('%Y-%m-%dT%H:%M:%S',time.localtime(time.time()))
        fetchedDate = date
        lastUsed = date        
        orginalContent = source.content
        
        
        if (urlparse(href).netloc == 'americannews.com'):              
            if (soup.findAll('h1')):
                for title in soup.findAll('h1'):
                    title = title.text
            else:
                title = 'Unknown'        
            if (soup.find('time',{'class': 'rpwe-time published'})):
                createdDate = soup.find('time',{'class': 'rpwe-time published'}).text
                createdDate = (parser.parse(createdDate)).isoformat()
            else: 
                createdDate = None  

            if (soup.findAll('b')):
                for author in soup.findAll('b'):
                    author = author.text
            else:
                author = 'Unknown'
        
        elif (urlparse(href).netloc == 'www.activistpost.com'):
            if (soup.findAll('span', {'class': 'entry-meta-date updated'})):
                for createdDate in soup.findAll('span', {'class': 'entry-meta-date updated'}):
                    createdDate = (parser.parse(createdDate.text)).isoformat()
            else:
                createdDate = None

            if (soup.findAll('h1', {'class': 'entry-title'})):
                for title in soup.findAll('h1', {'class': 'entry-title'}):
                    title = title.text
            else:
                title = 'Unknown'
            
            for authors in soup.findAll('p'):
                if (authors.findAll('a')):
                    for author in authors.findAll('a'):
                        author = author.text
                else: 
                    author = 'Unknown'
        
        
        
        elif (urlparse(href).netloc == 'www.thedailysheeple.com'):
            if (soup.findAll('time', {'class': 'entry-date'})):
                for createdDate in soup.findAll('time', {'class': 'entry-date'}):
                    createdDate = (parser.parse(createdDate.text)).isoformat()
            else:
                createdDate = None

                
            if (soup.findAll('h1', {'class': 'entry-title'})):
                for title in soup.findAll('h1', {'class': 'entry-title'}):
                    title = title.text
            else:
                title = 'Unknown'
            
            if (soup.findAll('span', {'class': 'author vcard'})):
                for author in soup.findAll('span', {'class': 'author vcard'}):
                    author = author.text
            else: 
                author = 'Unknown'
            
        
        
        elif (urlparse(href).netloc == 'waterfordwhispersnews.com'):
            if (soup.findAll('p', {'class': 'byline byline-left '})):
                for createdDate in soup.findAll('p', {'class': 'byline byline-left '}):
                    # createdDate = (parser.parse(createdDate.text)).isoformat()
                    createdDate = None
            else:
                createdDate = None
                
            if (soup.findAll('h1', {'class': 'entry-title'})):
                for title in soup.findAll('h1', {'class': 'entry-title'}):
                    title = title.text
            else:
                title = 'Unknown'
            
            if (soup.findAll('span', {'class': 'author vcard'})):
                for author in soup.findAll('span', {'class': 'author vcard'}):
                    author = author.text
            else: 
                author = 'Unknown'

        
        
        elif (urlparse(href).netloc == 'www.clickhole.com'):

            if (soup.findAll('div', {'class': 'pub_date'})):
                for createdDate in soup.findAll('div', {'class': 'pub_date'}):
                    createdDate = (parser.parse(createdDate.text)).isoformat()
            else:
                createdDate = None

            if (soup.findAll('h1', {'class': 'headline'})):
                for title in soup.findAll('h1', {'class': 'headline'}):
                    title = title.text
            else:
                title = 'Unknown'

            if (soup.findAll('span', {'class': 'author vcard'})):
                for author in soup.findAll('span', {'class': 'author vcard'}):
                    author = author.text
            else: 
                author = 'Unknown'

        
        elif (urlparse(href).netloc == 'theonion.com'):
            if (soup.findAll('span', {'class': 'content-published-mobile'})):
                for createdDate in soup.findAll('span', {'class': 'content-published-mobile'}):
                    createdDate = (parser.parse(createdDate.text)).isoformat()
            else:
                createdDate = None
            
            if (soup.findAll('header', {'class': 'content-header'})):
                for title in soup.findAll('header', {'class': 'content-header'}):
                    title = title.text
            else:
                title = 'Unknown'
            
            if (soup.findAll('span', {'class': 'author vcard'})):
                for author in soup.findAll('span', {'class': 'author vcard'}):
                    author = author.text
            else: 
                author = 'Unknown'
                        
        else:
            title = 'Unknown'
            author = 'Unknown'
            createdDate = None
    else:
        print ('can not get connent')
        return
    news = News(article, title, author, orginalContent, createdDate, date, href, date)
    scoring(news)
    
link = [
        'thedailysheeple.com',
#         'waterfordwhispersnews.com',
#         'clickhole.com',
#         'americannews.com',
#         'activistpost.com',
#         'theonion.com'
        ]

get_news_firstpage(link)


http://www.thedailysheeple.com/our-political-parties-are-obsolete_072017
********total:********** 5
http://www.thedailysheeple.com/palm-beach-sheriffs-deputy-on-paid-leave-after-child-porn-charges_072017
********total:********** 6
http://www.thedailysheeple.com/recordings-reveal-fbi-gave-man-a-rifle-urged-him-to-carry-out-mass-shooting-to-defend-islam_072017
********total:********** 5
http://www.futuremoneytrends.com/ag
********total:********** 5
http://www.thedailysheeple.com/kurdish-militia-commander-clashes-with-turkey-may-come-within-days_072017
********total:********** 6
http://www.thedailysheeple.com/study-finds-temperature-adjustments-account-for-nearly-all-of-the-warming-in-climate-data_072017
********total:********** 6
http://www.thedailysheeple.com/exorcism-business-is-booming-could-cern-be-to-blame-video_072017
********total:********** 6
http://www.thedailysheeple.com/fbi-allowed-to-hide-details-of-secret-911-report-judge_072017
********total:********** 5
http://www.thedaily